In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from feature_engine.outliers import Winsorizer
import joblib, pickle
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression


In [2]:
#Business Objective: Develop a Simple Linear Regression model correlating calorie consumption with weight gain to formulate personalized diet plans for individuals, aiming at effective weight management strategies.

In [3]:
calories = pd.read_csv('calories_consumed.csv')
calories.columns = ['wg','c']

calories

wg     c
0    108  1500
1    200  2300
2    900  3400
3    200  2200
4    300  2500
5    110  1600
6    128  1400
7     62  1900
8    600  2800
9   1100  3900
10   100  1670
11   150  1900
12   350  2700
13   700  3000

In [4]:
calories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   wg      14 non-null     int64
 1   c       14 non-null     int64
dtypes: int64(2)
memory usage: 356.0 bytes


In [5]:
calories.describe()

wg            c
count    14.000000    14.000000
mean    357.714286  2340.714286
std     333.692495   752.109488
min      62.000000  1400.000000
25%     114.500000  1727.500000
50%     200.000000  2250.000000
75%     537.500000  2775.000000
max    1100.000000  3900.000000

In [6]:
calories.isna().sum()

wg    0
c     0
dtype: int64

In [7]:
import sweetviz
calories_report = sweetviz.analyze([calories, 'calories'])
calories_report.show_html('calories_report.html')


                                             |                                             | [  0%]   00:00 ->…

Report calories_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
x = pd.DataFrame(calories['wg'])
y = pd.DataFrame(calories['c'])

In [9]:
num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy ='mean'))])
num_preprocessor = ColumnTransformer(transformers=[('impute', num_pipeline, x.columns)])
impute_data = num_preprocessor.fit(x)
impute_df = pd.DataFrame(impute_data.transform(x), columns = x.columns)

In [10]:
winsor = Winsorizer(capping_method='iqr',
                   tail = 'both',
                   fold = 1.5,
                   variables = list(x.columns))
outlier_pipeline = Pipeline([('wins', winsor)])
outlier_data = outlier_pipeline.fit(impute_df)
outlier_df = pd.DataFrame(outlier_data.transform(impute_df), columns = x.columns)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scale_pipeline = Pipeline([('scale', MinMaxScaler())])
scale_preprocessor = ColumnTransformer(transformers=[('scale', scale_pipeline, x.columns)])
scale_data = scale_preprocessor.fit(outlier_df)
scale_df = pd.DataFrame(scale_data.transform(outlier_df), columns = x.columns)

In [12]:
joblib.dump(impute_data, 'impute_SLR_task1')
joblib.dump(outlier_data, 'outlier_SLR_task1')
joblib.dump(scale_data, 'scale_SLR_task1')

['scale_SLR_task1']

In [17]:
# # Linear Regression using statsmodels package
import statsmodels.formula.api as smf


In [20]:
model = smf.ols('c ~wg', data = calories).fit()
model.summary()

C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      c   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     104.3
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           2.86e-07
Time:                        16:27:15   Log-Likelihood:                -96.170
No. Observations:                  14   AIC:                             196.3
Df Residuals:                      12   BIC:                             197.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1577.2007    100.541     15.687      0.000    1358.141    1796.260
wg             2.1344      0.209     10.211      0.000       1.679       2.590
==============================================================================
Omnibus:                        0.254   Durbin-Watson:                   2.308
Prob(Omnibus):                  0.881   Jarque-Bera (JB):                0.425
Skew:                          -0.098   Prob(JB):                        0.808
Kurtosis:                       2.169   Cond. No.                         719.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
pred1 = model.predict(pd.DataFrame(calories['wg']))

In [23]:
pred1

0     1807.718381
1     2004.085294
2     3498.181364
3     2004.085294
4     2217.527589
5     1811.987227
6     1850.406841
7     1709.534925
8     2857.854477
9     3925.065955
10    1790.642998
11    1897.364146
12    2324.248737
13    3071.296772
dtype: float64

In [24]:
# Error calculation (error = AV - PV)
res1 = calories.c- pred1


In [25]:
res1

0    -307.718381
1     295.914706
2     -98.181364
3     195.914706
4     282.472411
5    -211.987227
6    -450.406841
7     190.465075
8     -57.854477
9     -25.065955
10   -120.642998
11      2.635854
12    375.751263
13    -71.296772
dtype: float64

In [26]:
res_sqr1 = res1 * res1
mse1 = np.mean(res_sqr1)
rmse1 = np.sqrt(mse1)
rmse1


232.83350070960887

In [27]:
# # Model Tuning with Transformations
# ## Log Transformation


In [28]:
model2 = smf.ols('c ~ np.log(wg)', data = calories).fit()
model2.summary()

C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      c   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     86.04
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           8.02e-07
Time:                        16:32:59   Log-Likelihood:                -97.363
No. Observations:                  14   AIC:                             198.7
Df Residuals:                      12   BIC:                             200.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1911.1244    464.188     -4.117      0.001   -2922.503    -899.746
np.log(wg)   774.1735     83.462      9.276      0.000     592.326     956.021
==============================================================================
Omnibus:                        4.026   Durbin-Watson:                   3.009
Prob(Omnibus):                  0.134   Jarque-Bera (JB):                1.659
Skew:                           0.772   Prob(JB):                        0.436
Kurtosis:                       3.679   Cond. No.                         36.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
pred2 = model2.predict(pd.DataFrame(calories['wg']))

In [30]:
pred2

0     1713.657759
1     2190.692769
2     3355.109707
3     2190.692769
4     2504.593132
5     1727.863176
6     1845.189099
7     1283.993883
8     3041.209345
9     3510.463652
10    1654.076556
11    1967.976918
12    2623.932511
13    3160.548724
dtype: float64

In [49]:
# Error calculation
res2 =calories.c - pred2
res_sqr2 = res2 * res2
mse2 = np.mean(res_sqr2)
rmse2 = np.sqrt(mse2)
rmse2


253.5580403936626

In [32]:
# ## Exponential transformation


In [33]:
model3 = smf.ols('np.log(c) ~ wg', data = calories).fit()
model3.summary()

C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              np.log(c)   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     50.40
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           1.25e-05
Time:                        16:37:39   Log-Likelihood:                 8.3130
No. Observations:                  14   AIC:                            -12.63
Df Residuals:                      12   BIC:                            -11.35
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.4068      0.058    128.367      0.000       7.281       7.533
wg             0.0009      0.000      7.100      0.000       0.001       0.001
==============================================================================
Omnibus:                        0.488   Durbin-Watson:                   2.068
Prob(Omnibus):                  0.783   Jarque-Bera (JB):                0.559
Skew:                          -0.317   Prob(JB):                        0.756
Kurtosis:                       2.254   Cond. No.                         719.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
pred3 = model3.predict(pd.DataFrame(calories['wg']))

In [35]:
pred3

0     7.498820
1     7.577174
2     8.173348
3     7.577174
4     7.662342
5     7.500523
6     7.515853
7     7.459642
8     7.917845
9     8.343684
10    7.492006
11    7.534590
12    7.704926
13    8.003013
dtype: float64

In [36]:
pred3_c = np.exp(pred3)
pred3_c

0     1805.909334
1     1953.101471
2     3545.194617
3     1953.101471
4     2126.731695
5     1808.988061
6     1836.933870
7     1736.526880
8     2745.847366
9     4203.546672
10    1793.646733
11    1871.676808
12    2219.252162
13    2989.952498
dtype: float64

In [48]:
res3 = calories.c -pred3_c
res_sqr3 = res3 * res3
mse3 = np.mean(res_sqr3)
rmse3 = np.sqrt(mse3)
rmse3


272.4207117048494

In [38]:
# ## Polynomial transformation 


In [40]:
x.sort_values(by=['wg'], axis =0, inplace = True)

In [44]:
model4 = smf.ols('np.log(c) ~ wg + I(wg*wg)', data = calories).fit()
model4.summary()


C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              np.log(c)   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     31.55
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           2.78e-05
Time:                        16:48:00   Log-Likelihood:                 10.125
No. Observations:                  14   AIC:                            -14.25
Df Residuals:                      11   BIC:                            -12.33
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.2892      0.084     86.724      0.000       7.104       7.474
wg             0.0017      0.000      3.560      0.004       0.001       0.003
I(wg * wg) -7.689e-07   4.27e-07     -1.803      0.099   -1.71e-06     1.7e-07
==============================================================================
Omnibus:                        0.744   Durbin-Watson:                   2.647
Prob(Omnibus):                  0.689   Jarque-Bera (JB):                0.721
Skew:                          -0.394   Prob(JB):                        0.697
Kurtosis:                       2.217   Cond. No.                     1.01e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [45]:
pred4 = model4.predict(pd.DataFrame(calories))
pred4

0     7.461414
1     7.593995
2     8.176497
3     7.593995
4     7.723342
5     7.464435
6     7.491343
7     7.390243
8     8.019118
9     8.204529
10    7.449270
11    7.523555
12    7.782249
13    8.086955
dtype: float64

In [46]:
pred4_c = np.exp(pred4)

In [50]:
# Error calculation
res4 = calories.c -pred4_c
res_sqr4 = res4 * res4
mse4 = np.mean(res_sqr4)
rmse4 = np.sqrt(mse4)
rmse4


240.8277757040726

In [52]:
# ### Choose the best model using RMSE
data = {"MODEL":pd.Series(["SLR", "Log model", "Exp model", "Poly model"]), "RMSE":pd.Series([rmse1, rmse2, rmse3, rmse4])}

table_rmse = pd.DataFrame(data)

table_rmse


MODEL        RMSE
0         SLR  232.833501
1   Log model  253.558040
2   Exp model  272.420712
3  Poly model  240.827776

In [53]:
# # Evaluate the best model
# Data Split


In [54]:
train, test = train_test_split(calories, test_size = 0.2, random_state = 0)


In [55]:
# Fit the best model on train data
final_model = smf.ols('c ~ wg', data = train).fit()


In [56]:
final_pred = final_model.predict(test)

In [57]:
# Model Evaluation on Test data
test_res = test.c -pred1
test_sqrs = test_res * test_res
test_mse = np.mean(test_sqrs)
test_rmse = np.sqrt(test_mse)

test_rmse


308.76319152587996